# 5. LangChain Expression Language(LCEL) 심층 해설


In [9]:
!pip install dotenv

In [11]:
import os
# from google.colab import userdata
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

In [12]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 langchain-community==0.3.0 pydantic==2.10.6

  Using cached typing_extensions-4.15.0-py3-none-any.whl.metadata (3.3 kB)
INFO: pip is looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
  Using cached requests_toolbelt-1.0.0-py2.py3-none-any.w

## 5.1. Runnable과 RunnableSequence―LCEL의 가장 기본적인 구성 요소  

LECL을 잘 이해할 수 있도록 먼저 기존에 사용된 Runnable을 상속한 클래스를 모두 invoke

In [13]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [ ]:
# 순서대로 invoke
prompt_value = prompt.invoke({"dish": "카레"})
ai_message = model.invoke(prompt_value)
output = output_parser.invoke(ai_message)

print(output)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 피망, 버섯, 완두콩 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣어 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다.

5. **물 붓기**: 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불로 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞은 후, 다시 10분 정도 끓입니다. 필요에 따라 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 하루 정도 숙성시키면 더욱 맛있습니다.
- 다양한 채소나 해산물을 추가하여 나만의 카레를 만들어 보세요.

맛있게 드세요!


In [16]:
# LCEL에서는 Runnable을 | 연산자로 연결할 수 있음
chain = prompt | model | output_parser

In [17]:
output = chain.invoke({"dish": "카레"})
print(output)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 선택 재료도 적당한 크기로 썰어줍니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다. 이때 마늘과 생강을 추가하면 향이 더 좋아집니다.

5. **물 붓기**: 모든 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불에서 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞은 후, 다시 10분 정도 끓입니다. 필요에 따라 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 하루 정도 숙성시키면 더욱 맛있습니다.
- 다양한 재료를 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!


### Runnable의 실행 방법―invoke・stream・batch


In [18]:
chain = prompt | model | output_parser

for chunk in chain.stream({"dish": "카레"}):
    print(chunk, end="", flush=True)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 선택 재료가 있다면 함께 손질합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 볶아줍니다. 양파가 투명해질 때까지 볶습니다.

3. **고기 추가**: 손질한 고기를 넣고 겉면이 익을 때까지 볶습니다. 이때 마늘과 생강을 추가하면 향이 더 좋습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다. 다른 채소도 이 단계에서 추가할 수 있습니다.

5. **물 붓기**: 모든 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불에서 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞어줍니다. 원하는 농도에 따라 물을 추가할 수 있습니다. 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 5분 정도 뜸을 들인 후 그릇에 담아냅니다.

### 서빙
밥과 함께 서빙하면 좋습니다. 또한, 피클이나 요거트와 함께 곁들여 먹으면 더욱 맛있습니다.

맛있게 드세요!

In [19]:
chain = prompt | model | output_parser

outputs = chain.batch([{"dish": "카레"}, {"dish": "우동"}])
print(outputs)

['카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.\n\n### 재료\n- 고기 (닭고기, 소고기, 돼지고기 등) 300g\n- 양파 1개\n- 감자 1개\n- 당근 1개\n- 카레 가루 2-3 큰술\n- 코코넛 밀크 또는 물 2컵\n- 식용유 2 큰술\n- 소금, 후추 약간\n- 마늘 2쪽 (다진 것)\n- 생강 1 작은술 (다진 것)\n- 선택 재료: 피망, 버섯, 완두콩 등\n\n### 조리 방법\n1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다.\n\n2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 투명해질 때까지 볶습니다.\n\n3. **마늘과 생강 추가**: 다진 마늘과 생강을 넣고 향이 날 때까지 볶습니다.\n\n4. **고기 추가**: 손질한 고기를 넣고 겉면이 익을 때까지 볶습니다.\n\n5. **채소 추가**: 감자와 당근을 넣고 잘 섞은 후, 카레 가루를 넣고 1-2분 더 볶아 향을 내줍니다.\n\n6. **액체 추가**: 코코넛 밀크 또는 물을 붓고, 소금과 후추로 간을 맞춥니다. 끓어오르면 불을 줄이고 뚜껑을 덮고 20-30분간 끓입니다. 중간에 저어주면 좋습니다.\n\n7. **마무리**: 원하는 경우 피망, 버섯, 완두콩 등을 추가하고 5-10분 더 끓입니다.\n\n8. **서빙**: 밥이나 난과 함께 따뜻하게 서빙합니다.\n\n맛있게 드세요! 카레는 취향에 따라 다양한 재료와 향신료를 추가하여 변형할 수 있습니다.', '우동은 일본의 전통적인 면 요리로, 쫄깃한 면과 다양한 재료가 어우러져 맛있는 국물 요리입니다. 아래는 기본적인 우동 레시피입니다.\n\n### 재료\n- 우동 면 200g\n- 물 4컵\n- 다시마 10cm 조각\n- 가쓰오부시 1컵\n- 간장 3큰술\n- 미림 2큰술\n- 소금 약간\n- 대파 (썰어서) 1대\n- 버섯 (표고버섯, 느타리버섯 등) 100g\n- 시금치 또는 청

Runnable 클래스를 상속한 클래스는 invoke, stream, batch와 같은 통일된 인터페이스를 호출할 수 있음
비동기 처리하는 ainvoke, astream, abatch 메서드도 제공

### LCEL의 "|"로 다양한 Runnable 연결하기


In [20]:
# Zero-shot CoT 예시
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [21]:
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자의 질문에 단계적으로 답변하세요."),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser

In [22]:
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "단계적으로 생각한 답변에서 결론만 추출하세요."),
        ("human", "{text}"),
    ]
)

summarize_chain = summarize_prompt | model | output_parser

In [ ]:
# 가능한 이유는 출력타입과 입력타입의 일관성 때문
cot_summarize_chain = cot_chain | summarize_chain
output = cot_summarize_chain.invoke({"question": "10 + 2 * 3"})
print(output)

결론: \(10 + 2 * 3 = 16\)입니다.


LangSmith를 통해 내부 동작 확인 가능

## 5.2. RunnableLambda―임의의 함수를 Runnable로 만들기
LLM 애플리케이션에서는 LLM의 응답에 대해 규칙 기반으로 추가 처리를 하거나 어떤 변환을 적용하고 싶은 경우에 RunnableLambda를 사용하면 LCEL의 Chain에 임의의 처리(함수)를 연결할 수 있음

In [ ]:
# LLM이 생성한 텍스트에 대해 소문자를 대문자로 변환하는 처리를 연결하는 Chain 예시
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("human", "{input}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [ ]:
from langchain_core.runnables import RunnableLambda


def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | RunnableLambda(upper)

ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### chain 데코레이터를 사용한 RunnableLambda 구현


In [ ]:
from langchain_core.runnables import chain


@chain
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | upper

ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### RunnableLambda 자동 변환


In [28]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | output_parser | upper

In [29]:
ai_message = chain.invoke({"input": "Hello!"})
print(ai_message)

HELLO! HOW CAN I ASSIST YOU TODAY?


### Runnable의 입력 타입과 출력 타입에 주의


In [30]:
def upper(text: str) -> str:
    return text.upper()


chain = prompt | model | upper

# 아래 코드를 실행하면 오류가 발생합니다
output = chain.invoke({"input": "Hello!"})

AttributeError: 'AIMessage' object has no attribute 'upper'

In [31]:
chain = prompt | model | StrOutputParser() | upper

In [32]:
output = chain.invoke({"input": "Hello!"})
print(output)

HELLO! HOW CAN I ASSIST YOU TODAY?


### (칼럼) 사용자 함수를 stream에 대응시키는 방법


In [33]:
from typing import Iterator


def upper(input_stream: Iterator[str]) -> Iterator[str]:
    for text in input_stream:
        yield text.upper()


chain = prompt | model | StrOutputParser() | upper

for chunk in chain.stream({"input": "Hello!"}):
    print(chunk, end="", flush=True)

HELLO! HOW CAN I ASSIST YOU TODAY?

## 5.3. RunnableParallel―여러 Runnable을 병렬로 처리하기


In [34]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
output_parser = StrOutputParser()

In [35]:
optimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 낙관주의자입니다. 사용자의 입력에 대해 낙관적인 의견을 제공하세요."),
        ("human", "{topic}"),
    ]
)
optimistic_chain = optimistic_prompt | model | output_parser

In [36]:
pessimistic_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 비관주의자입니다. 사용자의 입력에 대해 비관적인 의견을 제공하세요."),
        ("human", "{topic}"),
    ]
)
pessimistic_chain = pessimistic_prompt | model | output_parser

In [37]:
import pprint
from langchain_core.runnables import RunnableParallel

parallel_chain = RunnableParallel(
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
)

output = parallel_chain.invoke({"topic": "생성 AI의 진화에 관해"})
pprint.pprint(output)

{'optimistic_opinion': '생성 AI의 진화는 정말 흥미로운 주제입니다! 기술이 발전함에 따라 우리는 더욱 창의적이고 '
                       '혁신적인 도구를 얻게 되었고, 이는 다양한 분야에서 긍정적인 변화를 가져오고 있습니다. 예를 '
                       '들어, 예술, 음악, 글쓰기 등에서 AI가 사람들과 협력하여 새로운 형태의 창작물을 만들어내고 '
                       '있습니다. \n'
                       '\n'
                       '또한, 생성 AI는 교육, 의료, 비즈니스 등 여러 산업에서 효율성을 높이고, 사람들의 삶을 더 '
                       '편리하게 만들어주는 데 기여하고 있습니다. 앞으로의 발전은 더욱 놀라운 가능성을 열어줄 것이며, '
                       '우리는 이 기술이 어떻게 우리의 상상력을 확장하고, 새로운 기회를 창출할지 기대할 수 있습니다. '
                       '긍정적인 변화가 가득한 미래가 기다리고 있습니다!',
 'pessimistic_opinion': '생성 AI의 진화는 분명 흥미로운 주제이지만, 그 이면에는 여러 가지 우려가 존재합니다. 기술이 '
                        '발전함에 따라 우리는 더욱 정교한 AI를 보게 될 것이고, 이는 인간의 창의성과 노동을 대체할 '
                        '가능성이 큽니다. 결국, 많은 사람들이 일자리를 잃고, 사회적 불평등이 심화될 수 있습니다.\n'
                        '\n'
                        '또한, 생성 AI가 만들어내는 콘텐츠의 품질과 신뢰성에 대한 문제도 있습니다. 잘못된 정보나 '
                        '편향된 데이터로 인해 생성된 결과물은 사회에 부정적인 영향을 미칠

### RunnableParallel의 출력을 Runnable의 입력으로 연결하기


In [38]:
synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 객관적 AI입니다. 두 가지 의견을 종합하세요."),
        ("human", "낙관적 의견: {optimistic_opinion}\n비관적 의견: {pessimistic_opinion}"),
    ]
)

In [39]:
synthesize_chain = (
    RunnableParallel(
        {
            "optimistic_opinion": optimistic_chain,
            "pessimistic_opinion": pessimistic_chain,
        }
    )
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "생성 AI의 진화에 관해"})
print(output)

생성 AI의 진화는 흥미로운 주제이며, 긍정적인 변화와 우려가 공존하는 복합적인 상황입니다. 낙관적인 시각에서는 생성 AI가 창의적이고 혁신적인 도구로서 예술, 음악, 글쓰기 등 다양한 분야에서 사람들과 협력하여 새로운 창작물을 만들어내고, 교육, 의료, 비즈니스 등 여러 산업에서 효율성을 높이며 사람들의 삶을 편리하게 만들어줄 것이라고 기대하고 있습니다. 이러한 발전은 우리의 상상력을 확장하고 새로운 기회를 창출할 가능성을 열어줍니다.

반면, 비관적인 시각에서는 생성 AI의 발전이 인간의 창의성과 노동을 대체할 위험이 있으며, 기계가 만들어낸 콘텐츠에 의존하게 될 경우 인간의 독창성과 감성이 무시될 수 있다는 우려가 있습니다. 또한, 생성 AI가 잘못된 정보나 편향된 데이터를 기반으로 결과물을 생성할 경우 사회에 부정적인 영향을 미칠 수 있으며, 이는 신뢰를 무너뜨리고 혼란을 초래할 수 있습니다. 기술 발전에 따른 책임과 규제의 복잡성도 문제로 지적되고 있습니다.

결론적으로, 생성 AI의 진화는 많은 가능성을 제공하지만, 그 이면에는 심각한 위험과 도전이 존재합니다. 우리는 이 기술이 가져올 변화에 대해 균형 잡힌 시각을 유지하며, 긍정적인 측면과 함께 부정적인 결과를 진지하게 고려해야 할 필요가 있습니다.


### RunnableParallel 자동 변환


In [41]:
synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
    }
    | synthesize_prompt
    | model
    | output_parser
)

In [42]:
output = synthesize_chain.invoke({"topic": "생성 AI의 진화에 관해"})
print(output)

생성 AI의 진화는 흥미로운 주제이며, 긍정적인 변화와 우려가 동시에 존재합니다. 낙관적인 시각에서는 AI가 예술, 음악, 글쓰기 등 다양한 분야에서 창의적인 협업을 통해 혁신적인 도구로 자리 잡고 있으며, 교육, 의료, 비즈니스 등 여러 산업에서 효율성을 높이고 사람들의 삶을 편리하게 만들어주는 긍정적인 역할을 하고 있다고 강조합니다. 이러한 발전은 AI와 함께 더 나은 미래를 만들어갈 수 있는 가능성을 제시합니다.

반면, 비관적인 시각에서는 AI의 발전이 인간의 창의성과 노동 시장에 위협이 될 수 있으며, 많은 사람들이 일자리를 잃고 경제적 불평등이 심화될 위험이 있다고 경고합니다. 또한, AI가 생성하는 콘텐츠의 품질이 높아짐에 따라 정보의 신뢰성이 떨어질 수 있는 문제도 지적됩니다. 이러한 사회적, 윤리적 문제들은 우리가 감당하기 어려운 수준에 이를 수 있으며, 기술이 우리를 구원할 것이라는 희망이 환상에 불과할 수 있다는 우려가 있습니다.

결론적으로, 생성 AI의 진화는 기술적 진보를 가져오지만, 그에 따른 사회적, 윤리적 문제를 함께 고려해야 할 필요성이 있습니다. 긍정적인 가능성과 부정적인 우려를 균형 있게 바라보며, 지속적인 논의와 대응이 필요할 것입니다.


### RunnableLambda와의 조합―itemgetter를 사용한 예시
Python 표준 라이브러리에서 제공하는 함수. dict 등에서 값을 추출하는 함수를 쉽게 만들 수 있음

In [43]:
from operator import itemgetter

topic_getter = itemgetter("topic")
topic = topic_getter({"topic": "생성 AI의 진화에 관해"})
print(topic)

생성 AI의 진화에 관해


In [ ]:
from operator import itemgetter

synthesize_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "당신은 객관적 AI입니다. {topic}에 대한 두 가지 의견을 종합하세요.",
        ),
        (
            "human",
            "낙관적 의견: {optimistic_opinion}\n비관적 의견: {pessimistic_opinion}",
        ),
    ]
)

synthesize_chain = (
    {
        "optimistic_opinion": optimistic_chain,
        "pessimistic_opinion": pessimistic_chain,
        "topic": itemgetter("topic"), # RunnableLambda(itemgetter("topic"))로 자동 변환
    }
    | synthesize_prompt
    | model
    | output_parser
)

output = synthesize_chain.invoke({"topic": "생성 AI의 진화에 관해"})
print(output)

생성 AI의 진화에 대한 두 가지 의견은 기술의 발전이 가져오는 긍정적인 가능성과 그로 인해 발생할 수 있는 부정적인 결과를 잘 보여줍니다.

낙관적인 시각에서는 생성 AI가 창의적이고 유용한 도구로 자리 잡으며, 예술, 음악, 글쓰기 등 다양한 분야에서 사람들의 상상력을 자극하고 새로운 아이디어를 창출하는 데 기여할 수 있다고 강조합니다. 개인화된 경험을 제공할 수 있는 잠재력 덕분에 사용자 맞춤형 콘텐츠를 생성하여 더 많은 사람들에게 긍정적인 영향을 미칠 수 있다는 점도 주목할 만합니다. 이러한 발전은 우리의 삶을 풍요롭게 할 수 있는 많은 가능성을 열어줄 것으로 기대됩니다.

반면, 비관적인 시각에서는 생성 AI의 발전이 인간의 창의성과 노동 시장에 위협이 될 수 있다고 경고합니다. 자동화가 진행됨에 따라 일자리 상실과 경제적 불평등이 심화될 가능성이 있으며, 생성 AI가 만들어내는 콘텐츠의 품질과 신뢰성 문제도 심각한 우려로 지적됩니다. 잘못된 정보나 편향된 데이터로 인해 사회적 혼란이 가중될 수 있다는 점은 기술 발전의 이면에 존재하는 중요한 문제입니다.

결론적으로, 생성 AI의 진화는 기술적 진보를 가져오지만, 그로 인해 발생할 수 있는 부정적인 결과를 간과해서는 안 됩니다. 우리는 이 기술이 가져올 변화에 대해 균형 잡힌 시각을 가지고 준비해야 할 필요가 있습니다.


## 5.4. RunnablePassthrough―입력을 그대로 출력하기


In [45]:
import os
from google.colab import userdata

os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

ModuleNotFoundError: No module named 'google'

In [46]:
!pip install tavily-python==0.5.0

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 고려해 질문에 답하세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)

In [ ]:
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke("서울의 현재 날씨는?")
print(output)

### assign―RunnableParallel의 출력에 값 추가하기


In [ ]:
import pprint

chain = {
    "question": RunnablePassthrough(),
    "context": retriever,
} | RunnablePassthrough.assign(answer=prompt | model | StrOutputParser())

output = chain.invoke("서울의 현재 날씨는?")
pprint.pprint(output)

In [ ]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())

In [ ]:
output = chain.invoke("서울의 현재 날씨는?")
print(output)

#### <보충:pick>


In [ ]:
chain = (
    RunnableParallel(
        {
            "question": RunnablePassthrough(),
            "context": retriever,
        }
    )
    .assign(answer=prompt | model | StrOutputParser())
    .pick(["context", "answer"])
)

In [ ]:
output = chain.invoke("서울의 현재 날씨는?")
print(output)

### (칼럼) astream_events


In [ ]:
# Google Colab에서는 다음 코드의 "async" 부분에 "Use of "async" not allowed outside of async function"이라고 표시되지만, 오류 없이 실행할 수 있습니다

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

async for event in chain.astream_events("서울의 현재 날씨는?", version="v2"):
    print(event, flush=True)

In [ ]:
async for event in chain.astream_events("서울의 현재 날씨는?", version="v2"):
    event_kind = event["event"]

    if event_kind == "on_retriever_end":
        print("=== 검색 결과 ===")
        documents = event["data"]["output"]
        for document in documents:
            print(document)

    elif event_kind == "on_parser_start":
        print("=== 최종 출력 ===")

    elif event_kind == "on_parser_stream":
        chunk = event["data"]["chunk"]
        print(chunk, end="", flush=True)

### (칼럼) Chat history와 Memory


In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

chain = prompt | model | StrOutputParser()

In [ ]:
from langchain_community.chat_message_histories import SQLChatMessageHistory


def respond(session_id: str, human_message: str) -> str:
    chat_message_history = SQLChatMessageHistory(
        session_id=session_id, connection="sqlite:///sqlite.db"
    )

    ai_message = chain.invoke(
        {
            "chat_history": chat_message_history.get_messages(),
            "input": human_message,
        }
    )

    chat_message_history.add_user_message(human_message)
    chat_message_history.add_ai_message(ai_message)

    return ai_message

In [ ]:
from uuid import uuid4

session_id = uuid4().hex

output1 = respond(
    session_id=session_id,
    human_message="안녕하세요! 저는 존이라고 합니다!",
)
print(output1)

output2 = respond(
    session_id=session_id,
    human_message="제 이름을 알고 계신가요?",
)
print(output2)